In [ ]:
import pandas as pd
import NanobodyPaperPlotting
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#"globals"
lowBDFPCutoff = 0.75
highBDFPCutoff = 1

colors_decimal = [[120/255, 120/255, 120/255], [249/255, 29/255, 0/255], [32/255, 25/255, 250/255]]

#set plotting limits
#NanobodyPaperPlotting uses 10**n values, but sns.kdeplot uses n values
expressionLims = (10**0.25, 10**3)
BDFPPositiveBDFPSSCLims = (10**-1, 10**3)
BDFPNegativeBDFPSSCLims = (10**-4, 10**1)

expressionLims_kde = (0.25, 3)
BDFPPositiveBDFPSSCLims_kde = (-1, 3)
BDFPNegativeBDFPSSCLims_kde = (-4, 1)

mEosExpressionLabel = "mEos3 concentration (p.d.u.)"
BDFPExpressionLabel = "BDFP concentration (p.d.u.)"

In [ ]:
#data labeling
def labelsesA(data):
    # risingAmFRETCutoff = 0.09
    risingAmFRETMinExpression = 15
    
    highestAmFRETCutoff = 0.25
    
    data["label"] = 0
    
    indexesAboveExpressionCutoff = data["Acceptor/SSC"] >= risingAmFRETMinExpression
    # data.loc[indexesAboveExpressionCutoff & (data["AmFRET"] > risingAmFRETCutoff), "label"] = 1
    
    # data.loc[indexesAboveExpressionCutoff & (data["AmFRET"] > highestAmFRETCutoff), "label"] = 2
    data.loc[indexesAboveExpressionCutoff & (data["AmFRET"] > highestAmFRETCutoff), "label"] = 1

    return data
    

In [ ]:
#file helpers
def dataToPlotFromFilelist(files, title):
    """
    handles the single title filelist portion of titledFiles (parameter for DAmFRETRow_files)
    files should be a 1d list of files in population label order
    """
    data = pd.DataFrame()
    for i, file in enumerate(files):
        # populationData = DAmFRETClusteringTools.readDataToDF(file, minAmFRET=-10, maxAmFRET=10, minAmFRETPercentile=0, maxAmFRETPercentile=100, minAcceptorPercentile=0, maxAcceptorPercentile=100, xAxis="log(Acceptor/SSC)")
        populationData = NanobodyPaperPlotting.readDataToDF(file, minAmFRET=-10, maxAmFRET=10, minAmFRETPercentile=0, maxAmFRETPercentile=100, minAcceptorPercentile=0, maxAcceptorPercentile=100, xAxis="log(Acceptor/SSC)")
        populationData["label"] = i
        data = pd.concat([data, populationData])
        
    #These values were decided early on and they should not be function parameters!
    lowBDFPCutoff = 0.75
    highBDFPCutoff = 1
    
    dataLowBDFP = data[data["BDFP1.6-A"] <= (10 ** lowBDFPCutoff)]
    dataHighBDFP = data[data["BDFP1.6-A"] >= (10 ** highBDFPCutoff)]

    if title.lower() == "control" or title.lower() == "x3912b":
        return (dataLowBDFP, "bdfp-")
    else:
        return (dataHighBDFP, "BDFP+")

In [ ]:
def makeFPosPlot(xVals, yVals, ax, dataTitles):
    #FPos stands for "fraction positive" and is the proportion of cells that have high AmFRET values
    pointSize = 15
    alpha = 0.5
    
    for singleSourceYVals in yVals:
        ax.scatter(xVals, singleSourceYVals, s=pointSize, alpha=alpha)
    
    ax.set_xlabel(mEosExpressionLabel)
    ax.set_xscale("log")
    ax.set_xlim(expressionLims)
    ax.set_ylim((-0.1,1.1))

    ax.legend(dataTitles)

In [ ]:
sesAUngatedFiles = [
    ("Control", ["data/SesA/exported_plate4/A1.fcs"]),
    ("1x mEosNb", ["data/SesA/exported_plate5/G1.fcs"]),
    ("2x mEosNb", ["data/SesA/exported_plate5/H1.fcs"]),
    ("3x mEosNb", ["data/SesA/exported_plate6/A1.fcs"]),
    ("4x mEosNb", ["data/SesA/exported_plate6/B1.fcs"]),
    ("5x mEosNb", ["data/SesA/exported_plate6/C1.fcs"]),
    ("6x mEosNb", ["data/SesA/exported_plate6/D1.fcs"]),
]

In [ ]:
for title, files in sesAUngatedFiles:
    dataToPlot, _ = dataToPlotFromFilelist(files, title)
    dataToPlot = labelsesA(dataToPlot)

    NanobodyPaperPlotting.plotDAmFRETClusters(dataToPlot["Acceptor/SSC"], dataToPlot["AmFRET"], dataToPlot["label"], colors=colors_decimal, logX=True)
    

In [ ]:
#get data for sesA
numDivisions = 30
boundaries = np.logspace(expressionLims_kde[0], expressionLims_kde[1], numDivisions)
minPerBin = 75

meanExpressionInBins = []
for i, lowerBoundary in enumerate(boundaries[:-1]):
    upperBoundary = boundaries[i + 1]

    avgAccSSC = 10 ** ((np.log10(lowerBoundary) + np.log10(upperBoundary)) / 2)
    meanExpressionInBins.append(avgAccSSC)

pop1Summaries_sesA = [] #list of floats for EACH well
pop2Summaries_sesA = [] #list of floats for EACH well
pop12Summaries_sesA = [] #list of floats for EACH well

for title, files in sesAUngatedFiles:
    dataToPlot, _ = dataToPlotFromFilelist(files, title)
    dataToPlot = labelsesA(dataToPlot)

    pop1TempSummary = []
    pop2TempSummary = []
    pop12TempSummary = []
    
    for i, lowerBoundary in enumerate(boundaries[:-1]):
        upperBoundary = boundaries[i + 1]
    
        avgAccSSC = (lowerBoundary + upperBoundary) / 2
    
        dataInSlice = dataToPlot[(dataToPlot["Acceptor/SSC"] >= lowerBoundary) & (dataToPlot["Acceptor/SSC"] <= upperBoundary)]
        labels, labelCounts = np.unique(dataInSlice["label"], return_counts=True)
        labelCountDict = dict(zip(labels, labelCounts))

        if sum(labelCounts) < minPerBin: #don't trust any values because there isn't enough data
            pop1TempSummary.append(None)
            pop2TempSummary.append(None)
            pop12TempSummary.append(None)
        
        else:  
            defaultValue = 0 #if a label doesn't appear, there are no cells with that label
            # if 1 in labelCountDict:
            #     pop1TempSummary.append(labelCountDict[1] / sum(labelCounts))
            # else: #can trust the data, and it does not appear
            #     pop1TempSummary.append(0)
            # if 2 in labelCountDict:
            #     pop2TempSummary.append(labelCountDict[2] / sum(labelCounts))
            # else:
            #     pop2TempSummary.append(0)
            
            # if 1 in labelCountDict and 2 in labelCountDict:
            #     pop12TempSummary.append((labelCountDict[1] + labelCountDict[2]) / sum(labelCounts))
            # else:
            #     pop12TempSummary.append(0)
            
            pop1TempSummary.append(labelCountDict.get(1, defaultValue) / sum(labelCounts))
            pop2TempSummary.append(labelCountDict.get(2, defaultValue) / sum(labelCounts))
            pop12TempSummary.append((labelCountDict.get(1, defaultValue) + labelCountDict.get(2, defaultValue)) / sum(labelCounts))
            
            
    pop1Summaries_sesA.append(pop1TempSummary)
    pop2Summaries_sesA.append(pop2TempSummary)
    pop12Summaries_sesA.append(pop12TempSummary)

In [ ]:
SesAFig = plt.Figure((3*4, 3), dpi=300)
SesAAxs = SesAFig.subplots(1,3)

makeFPosPlot(meanExpressionInBins, pop1Summaries_sesA, SesAAxs[0], [title for title, files in sesAUngatedFiles])
makeFPosPlot(meanExpressionInBins, pop2Summaries_sesA, SesAAxs[1], [title for title, files in sesAUngatedFiles])
makeFPosPlot(meanExpressionInBins, pop12Summaries_sesA, SesAAxs[2], [title for title, files in sesAUngatedFiles])

SesAAxs[0].set_ylabel("Proportion red")
SesAAxs[1].set_ylabel("Proportion blue")
SesAAxs[2].set_ylabel("Proportion red or blue")

SesAAxs[0].get_legend().remove()
SesAAxs[1].get_legend().remove()

legend = SesAAxs[2].get_legend()
#inspired by https://stackoverflow.com/questions/23238041/move-and-resize-legends-box-in-matplotlib
# Get the bounding box of the original legend
bb = legend.get_bbox_to_anchor().transformed(SesAAxs[2].transAxes.inverted()) 

# Change to location of the legend. 
xOffset = 0.6
bb.x0 += xOffset
bb.x1 += xOffset
legend.set_bbox_to_anchor(bb, transform = SesAAxs[2].transAxes)

leftShift = 0
SesAAxs[0].text(-0.25 - leftShift, 0.5, "SesA", transform=SesAAxs[0].transAxes, ha="center", va="center", rotation_mode="anchor", rotation=90, fontsize=10)


SesAFig.tight_layout()
SesAFig

In [ ]:
#for final - SESA LABELING WILL BE DIFFERENT!!!! AAAAAAH!

In [ ]:
ASCFiles = [
    ("Control", ["data/ASC/gated/gate0/A11_x3912b.fcs",
              "data/ASC/gated/gate1/A11_x3912b.fcs",
                "data/ASC/gated/gate2/A11_x3912b.fcs"]),
    ("1x mEosNb", ["data/ASC/gated/gate0/G11.fcs",
              "data/ASC/gated/gate1/G11.fcs",
                  "data/ASC/gated/gate2/G11.fcs"]),
    ("2x mEosNb", ["data/ASC/gated/gate0/H11.fcs",
              "data/ASC/gated/gate1/H11.fcs",
                  "data/ASC/gated/gate2/H11.fcs"]),
    ("3x mEosNb", ["data/ASC/gated/gate0/A11.fcs",
              "data/ASC/gated/gate1/A11.fcs",
                  "data/ASC/gated/gate2/A11.fcs"]),
    ("4x mEosNb", ["data/ASC/gated/gate0/B11.fcs",
              "data/ASC/gated/gate1/B11.fcs",
                  "data/ASC/gated/gate2/B11.fcs"]),
    ("5x mEosNb", ["data/ASC/gated/gate0/C11.fcs",
              "data/ASC/gated/gate1/C11.fcs",
                  "data/ASC/gated/gate2/C11.fcs"]),
    ("6x mEosNb", ["data/ASC/gated/gate0/D11.fcs",
              "data/ASC/gated/gate1/D11.fcs",
                  "data/ASC/gated/gate2/D11.fcs"])
]

In [ ]:
#get data for ASC
numDivisions = 30
boundaries = np.logspace(expressionLims_kde[0], expressionLims_kde[1], numDivisions)
# minPerBin = 75

meanExpressionInBins = []
for i, lowerBoundary in enumerate(boundaries[:-1]):
    upperBoundary = boundaries[i + 1]

    avgAccSSC = 10 ** ((np.log10(lowerBoundary) + np.log10(upperBoundary)) / 2)
    meanExpressionInBins.append(avgAccSSC)

pop1Summaries_ASC = [] #list of floats for EACH well
pop2Summaries_ASC = [] #list of floats for EACH well
pop12Summaries_ASC = [] #list of floats for EACH well

for title, files in ASCFiles:
    dataToPlot, _ = dataToPlotFromFilelist(files, title)

    pop1TempSummary = []
    pop2TempSummary = []
    pop12TempSummary = []
    
    for i, lowerBoundary in enumerate(boundaries[:-1]):
        upperBoundary = boundaries[i + 1]
    
        avgAccSSC = (lowerBoundary + upperBoundary) / 2
    
        dataInSlice = dataToPlot[(dataToPlot["Acceptor/SSC"] >= lowerBoundary) & (dataToPlot["Acceptor/SSC"] <= upperBoundary)]
        labels, labelCounts = np.unique(dataInSlice["label"], return_counts=True)
        labelCountDict = dict(zip(labels, labelCounts))

        if sum(labelCounts) < minPerBin: #don't trust any values because there isn't enough data
            pop1TempSummary.append(None)
            pop2TempSummary.append(None)
            pop12TempSummary.append(None)
        
        else:  
            defaultValue = 0 #if a label doesn't appear, there are no cells with that label
            # if 1 in labelCountDict:
            #     pop1TempSummary.append(labelCountDict[1] / sum(labelCounts))
            # else: #can trust the data, and it does not appear
            #     pop1TempSummary.append(0)
            # if 2 in labelCountDict:
            #     pop2TempSummary.append(labelCountDict[2] / sum(labelCounts))
            # else:
            #     pop2TempSummary.append(0)
            
            # if 1 in labelCountDict and 2 in labelCountDict:
            #     pop12TempSummary.append((labelCountDict[1] + labelCountDict[2]) / sum(labelCounts))
            # else:
            #     pop12TempSummary.append(0)
            
            pop1TempSummary.append(labelCountDict.get(1, defaultValue) / sum(labelCounts))
            pop2TempSummary.append(labelCountDict.get(2, defaultValue) / sum(labelCounts))
            pop12TempSummary.append((labelCountDict.get(1, defaultValue) + labelCountDict.get(2, defaultValue)) / sum(labelCounts))
            
    pop1Summaries_ASC.append(pop1TempSummary)
    pop2Summaries_ASC.append(pop2TempSummary)
    pop12Summaries_ASC.append(pop12TempSummary)

In [ ]:
ASCFig = plt.Figure((3*4, 3), dpi=300)
ASCAxs = ASCFig.subplots(1,3)

makeFPosPlot(meanExpressionInBins, pop1Summaries_ASC, ASCAxs[0], [title for title, files in ASCFiles])
makeFPosPlot(meanExpressionInBins, pop2Summaries_ASC, ASCAxs[1], [title for title, files in ASCFiles])
makeFPosPlot(meanExpressionInBins, pop12Summaries_ASC, ASCAxs[2], [title for title, files in ASCFiles])

ASCAxs[0].set_ylabel("Proportion red")
ASCAxs[1].set_ylabel("Proportion blue")
ASCAxs[2].set_ylabel("Proportion red or blue")

ASCAxs[0].get_legend().remove()
ASCAxs[1].get_legend().remove()

legend = ASCAxs[2].get_legend()
#inspired by https://stackoverflow.com/questions/23238041/move-and-resize-legends-box-in-matplotlib
# Get the bounding box of the original legend
bb = legend.get_bbox_to_anchor().transformed(ASCAxs[2].transAxes.inverted()) 

# Change to location of the legend. 
xOffset = 0.6
bb.x0 += xOffset
bb.x1 += xOffset
legend.set_bbox_to_anchor(bb, transform = ASCAxs[2].transAxes)


leftShift = 0.0
ASCAxs[0].text(-0.25 - leftShift, 0.5, "ASC", transform=ASCAxs[0].transAxes, ha="center", va="center", rotation_mode="anchor", rotation=90, fontsize=10)

ASCFig.tight_layout()
ASCFig

In [ ]:
#only red and blue is required for final version

In [ ]:
combinedFig = plt.Figure((2*4, 3), dpi=300)
combinedAxs = combinedFig.subplots(1,2)

makeFPosPlot(meanExpressionInBins, pop12Summaries_ASC, combinedAxs[0], [title for title, files in ASCFiles])
makeFPosPlot(meanExpressionInBins, pop1Summaries_sesA, combinedAxs[1], [title for title, files in sesAUngatedFiles])

combinedAxs[0].set_title("ASC")
combinedAxs[1].set_title("SesA")

combinedAxs[0].set_ylabel("Proportion high FRET")
combinedAxs[1].set_ylabel("Proportion high FRET")

combinedAxs[0].get_legend().remove()

legend = combinedAxs[1].get_legend()
#inspired by https://stackoverflow.com/questions/23238041/move-and-resize-legends-box-in-matplotlib
# Get the bounding box of the original legend
bb = legend.get_bbox_to_anchor().transformed(combinedAxs[1].transAxes.inverted()) 

# Change to location of the legend. 
xOffset = 0.6
bb.x0 += xOffset
bb.x1 += xOffset
legend.set_bbox_to_anchor(bb, transform = combinedAxs[1].transAxes)

combinedFig.tight_layout() 
combinedFig

In [ ]:
# combinedFig.savefig("plots/ASC_SesA_fpos.png")